In [ ]:
!pip install pydub
!pip install pytube
!pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import os
from collections import defaultdict
from pyannote.audio import Pipeline
from pyannote.core import Annotation, Segment
from transformers import pipeline
from google.colab import userdata

# Function to download and convert YouTube audio to WAV format
def download_and_convert_audio(video_url):
    yt = YouTube(video_url)
    stream = yt.streams.filter(only_audio=True).first()
    audio_file_path_mp4 = yt.title + ".mp4"
    audio_file_path_wav = yt.title + ".wav"

    stream.download(filename=audio_file_path_mp4)

    audio = AudioSegment.from_file(audio_file_path_mp4)
    audio = audio.set_frame_rate(44100).set_channels(1)
    audio.export(audio_file_path_wav, format="wav")

    os.remove(audio_file_path_mp4)
    return audio_file_path_wav

# Function to perform speaker diarization
def perform_speaker_diarization(audio_file_path, hf_token):
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token)
    result = pipeline(audio_file_path)

    speaker_segments = defaultdict(list)
    for segment, _, label in result.itertracks(yield_label=True):
        speaker_segments[label].append((segment.start, segment.end))

    total_speakers = len(speaker_segments)

    audio = AudioSegment.from_file(audio_file_path)
    speaker_audio = {}
    for speaker, segments in speaker_segments.items():
        combined = AudioSegment.empty()
        for start_time, end_time in segments:
            start_ms = int(start_time * 1000)
            end_ms = int(end_time * 1000)
            combined += audio[start_ms:end_ms]
        speaker_audio[speaker] = combined

    files = []
    for speaker, segment in speaker_audio.items():
        file_name = f"{speaker}.wav"
        segment.export(file_name, format="wav")
        files.append(file_name)

    return total_speakers, files

# Function to classify speakers by gender
def classify_speakers(files):
    pipe = pipeline("audio-classification", model="alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech")
    predictions = []

    for file in files:
        result = pipe(f"./{file}")
        predictions.append((file, result))

    return predictions

# Main function to process the video URL and provide required information
def process_video(video_url, hf_token):
    audio_file_path = download_and_convert_audio(video_url)
    total_speakers, files = perform_speaker_diarization(audio_file_path, hf_token)

    predictions = classify_speakers(files)

    # Count male and female speakers
    male_count = 0
    female_count = 0
    for file, prediction in predictions:
        highest_prediction = max(prediction, key=lambda x: x['score'])
        if highest_prediction['label'] == 'male':
            male_count += 1
        elif highest_prediction['label'] == 'female':
            female_count += 1

    return {
        "total_speakers": total_speakers,
        "male_speakers": male_count,
        "female_speakers": female_count
    }

# Example usage
if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=RjIxvwHZ3i0"
    hf_token = userdata.get('HF_KEY')  # Replace with your Hugging Face token
    result = process_video(video_url, hf_token)
    print(result)


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

{'total_speakers': 3, 'male_speakers': 1, 'female_speakers': 2}
